In [2]:
from numeric_calculus_mashup_package.Functions import Func
from numeric_calculus_mashup_package.Matrix import Matrix
from numeric_calculus_mashup_package.Vectors import Vector
from numeric_calculus_mashup_package.Linear_systems import Linear_System
from numeric_calculus_mashup_package.points_interpolation import Interpolate
from numeric_calculus_mashup_package.Linear_adjusts import Linear_adjust
import numeric_calculus_mashup_package.ploting_curves as plt_curves

import matplotlib.pyplot as plt
import sympy as sym
import numpy as np

In [57]:
class Analysis :
    def __init__(self, expression, args_info: dict, representation: str ='α') -> None:
        # args_info must be a dict like { arg_symbol: (value, error) }
        self.expr = expression
        self.repr = representation

        self.args_symbols = list(args_info.keys())
        self.args_vals = dict()
        self.args_errors = dict()

        for i, values in enumerate(args_info.values()):
            self.args_vals[self.args_symbols[i]] = values[0]
            self.args_errors[self.args_symbols[i]] = values[1]

    def __call__ (self, args_info: dict):
        self.args_symbols = list(args_info.keys())
        self.args_vals = dict()
        self.args_errors = dict()

        for i, values in enumerate(args_info.values()):
            self.args_vals[self.args_symbols[i]] = values[0]
            self.args_errors[self.args_symbols[i]] = values[1]

    def __repr__ (self) -> str:
        return self.repr

    def value_error_expression (self):

        quadratic_of_parcial_derivatives = [
            sym.Pow(self.expr.diff(arg) * self.args_errors[arg], 2, evaluate=False)
            if arg in self.expr.args else 0
            for arg in self.args_vals.keys()
        ]

        err_expression = sym.sqrt(sym.Add(*quadratic_of_parcial_derivatives))

        return err_expression
    
    def error (self) -> float:
        err_expr = self.value_error_expression()

        return err_expr.subs(self.args_vals)
    
    def value (self):
        return self.expr.subs(self.args_vals)

from math import pi

L, C, R, Omega_r,  delta_t, f = sym.symbols('L C R ωR Δt f')

Omega = 2 * pi * f

Pr = 0.5 * (
    (R * 16) /
    (sym.Pow(R, 2) + sym.Pow(
        (Omega * L) / 
        (1 - (sym.Pow(Omega, 2) * L * C))
    , 2))
)

Pr_max = 16 / (2 * R)

delta_Omega_r = 1 / (R * C)
Q = Omega_r * R * C


X_L = Omega * L
X_C = 1 / (Omega * C)

fi_exper = Omega * delta_t

fi_nom = sym.atan((X_C - X_L)/R)

f_R = Omega_r / (2 * pi)

args_values = {
    L: (24.5e-3, 1e-4),
    C: (4.6e-9, 1e-10),
    R: (562, 1),
    Omega_r: (94.2e3, 0.4e3),
    delta_t: (0, 0),
    f: (0, 0)
}

freq_values = [
    ('<valor f>', 0),
]

delta_t_values = [
     ('<valor delta_t>', '<erro delta t>')
]

table = [
    ['f', delta_t , 'fi_exper', 'fi_nom'],
]

anal_fi_exper = Analysis(fi_exper, args_values)
anal_fi_nom = Analysis(fi_nom, args_values)



for f_val, delta_t_val in zip(freq_values, delta_t_values):
    args_values[delta_t] = delta_t_val
    args_values[f] = f_val
    anal_fi_nom(args_values)
    anal_fi_exper(args_values)

    fi_exper = (anal_fi_exper.value(), anal_fi_exper.error())
    fi_nom = (anal_fi_nom.value(), anal_fi_nom.error())

    table.append([f_val, delta_t_val, fi_exper, fi_nom])

table_matrix = table

table_matrix


    

SympifyError: Sympify of expression 'could not parse '<valor delta_t>'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)